In [29]:
# Python calculation for the example and exercise from Lecture 4 notes
import numpy as np
import os
import math
import pandas as pd
import scipy.stats as stat
import scipy.interpolate
import statistics
from statistics import NormalDist

In [51]:
aapl = pd.read_excel("./data/hist_data.xlsm", sheet_name = 'AAPL')
msft = pd.read_excel("./data/hist_data.xlsm", sheet_name = 'MSFT')
f = pd.read_excel("./data/hist_data.xlsm", sheet_name = 'F')
bac = pd.read_excel("./data/hist_data.xlsm", sheet_name = 'BAC')
sofr_curve = pd.read_excel("./data/hist_data.xlsm", sheet_name = 'SofrCurve')

# Extract "T" as a separate array
T_array = sofr_curve["T"].values
df_transposed = sofr_curve[sofr_curve.columns[2:]].T
df_transposed.reset_index(inplace=True)
col_names = ['Date'] + list(sofr_curve['Tenor'])
df_transposed.columns = col_names

sofr_curve = df_transposed[['Date'] + list(df_transposed.columns[7:17])]
df_swap = pd.concat([df_transposed['Date'], sofr_curve[sofr_curve.columns[1:]].diff()],
                    axis=1)
df_stocks = pd.concat([aapl,
                        msft['Adj Close'],
                        f['Adj Close'],
                        bac['Adj Close']],axis=1)
df_stocks.columns = ['Date', 'aapl', 'msft', 'f', 'bac']
df_stocks[df_stocks.columns[1:]] = df_stocks[df_stocks.columns[1:]].pct_change()


df_returns = df_stocks.merge(df_swap,        # TODO change sofr curve to be modelled with change in rate rather than return
                        on=['Date'],
                        how='outer')\
                        .ffill()\
                        .drop(columns=['Date'])\
                        .dropna()

df_returns.head()

,aapl,msft,f,bac,1Y,2Y,3Y,4Y,5Y,6Y,7Y,8Y,9Y,10Y
1,-0.017543,-0.017059,0.002244,0.004439,0.000521,0.000439,0.000342,0.000296,0.000225,0.000124,0.000029,-0.000043,-0.000097,-0.000144
2,-0.037305,-0.035368,-0.025373,-0.003039,0.000234,0.000474,0.000404,0.000254,0.000153,0.000118,0.000109,0.000099,0.000090,0.000091
3,-0.042405,-0.026579,0.015314,-0.005542,0.000690,0.001098,0.001085,0.000957,0.000832,0.000761,0.000704,0.000624,0.000545,0.000495
4,-0.001947,0.033326,0.018854,0.025077,-0.000394,-0.000497,-0.000448,-0.000322,-0.000204,-0.000116,-0.000038,0.000045,0.000126,0.000192
5,0.003902,0.029270,0.014064,0.005980,0.000429,0.000536,0.000499,0.000413,0.000332,0.000313,0.000328,0.000342,0.000345,0.000339


## For Full Revaluation
$$\begin{aligned}
P_0 &= 100mio P_swap_0 + 1mio AAPL_0 + 1mio MSFT_0 + 1mio F + 1mio BAC_0     \\
P_1 &= 100mio P_swap_1 + 1mio AAPL_1 + 1mio MSFT_1 + 1mio F_1 + 1mio BAC_1   \\

\Delta_1 L &= P_1 - P_0  \\
&=  \left[ 1e8 S_{swap}(1) +  S_{aapl}(1)  +  S_{msft}(1)  +  S_{f}(1)  +  S_{bac}(1) \right] \\
    & \, \,- \left[ 1e8 S_{swap}(0) +  S_{aapl}(0)  +  S_{msft}(0)  +  S_{f}(0)  +  S_{bac}(0)  \right] \\
    &= \left[ 1e8 S_{swap}(1) - 1e8 S_{swap}(0) \right] + \left[  S_{aapl}(1) -  S_{aapl}(0) \right] + \left[  S_{msft}(1) -  S_{msft}(0) \right] \\
    & \,\, + \left[  S_{f}(1) -  S_{f}(0) \right]  + \left[  S_{bac}(1) -  S_{bac}(0) \right] \\         \\
\Delta_1 L    &= N \left[ \sum_{i=1}^T PV01_i \times \Delta R_i  \right] +  S_{aapl}(0) R_{aapl}^1  + S_{msft}(0)  R_{msft}^1  + S_{f}(0)  R_{f}^1  + S_{bac}(0) R_{bac}^1 \\         \\
\end{aligned}$$

### Pricing swap for full eval
compute changes in zero rate based on sample $(Z_{1} = Z_0 * (1+R_1))$ use new SOFT curve  to price swap at 4.2%


## For Sensitivity Analysis
$$\begin{aligned}
\Delta_1 L (\mu , \sigma^2) \\
\\
\mu \approx &100mio \mathbb{E}[\Delta_{swap}] + 1mio \mathbb{E}[\Delta_{aapl} + \Delta_{msft}  + \Delta_{f}  + \Delta_{bac}]        \\

\sigma^2 \approx &\text{Var}(100mio\Delta_{swap}) + \text{Var}(1mio [\Delta_{aapl} + \Delta_{msft}  + \Delta_{f}  + \Delta_{bac}])
\end{aligned}$$

### Pricing swap for sensitivity change 
get partial differential by changing one tenor by 1 bp then mark the change in PV.

$$
PV01_i = \frac{S(0,r_i + \Delta_{r_i}) - S(0,r_i)}{\Delta_{r_i}}
$$
For our model, we use $\Delta_{r_i} = 0.0001$
$$\begin{aligned}
\Delta PV &= N \left[ \sum_{i=1}^T PV01_i \times \Delta_{r_i} \right]  \\ 

\Delta PV &= N \left[ \sum_{i=1}^T PV01_i \times  R_{r_i} PV_0^i \right]  

\end{aligned}$$
where N is the notional of swap and $\Delta_{r_i} =  R_{r_i} PV_0^i$

---

# Useful functions and constants

1. Make a function to calculate payer swap
2. Get discount factors
3. Calculate initial value of swap
4. Calculate PV01:\
    a. change the value of each related zero rate by one bp an take note of PV change as partial derivative of PV for one bp change

In [52]:
def payer_swap_10y(ls_df, swap_rate):
    """
    Retun value of payer swap
    parameters
        ls_df: list of discount factors
        swap_rate: strike of swap
    fix_leg = sum of DF for 1y to 10 y
    flt_leg = 1 - D(0,T), since flt leg = sum of D(0,0) - D(0,1y) + D(0,1y) - D(0,2y) ... D(0,T)
    """
    fix_leg = sum(ls_df) * swap_rate * 1    # PVBP * Swap rate * day count fraction
    flt_leg = 1-ls_df[-1]

    return flt_leg - fix_leg
# initial value of swap
swap_rate = .042
ls_zero_rates = list(sofr_curve.iloc[-1][sofr_curve.columns[1:]].astype('float')) # get zero rates for 10 year swap
ls_df = [ np.exp(-r*(i+1)) for i, r in enumerate(ls_zero_rates) ] # calculate as discount factors
S_0 = payer_swap_10y(ls_df, swap_rate)

### Calculate PV01
Create list of zero rates. For each year, add a bp, save list of resultant DF. Calculate new payer swap PV, and difference from previous day. Save results to list

In [53]:
ls_pv01 = []

for i in range(len(ls_zero_rates)):
    ls_df = [r+.0001 if i==j else r for j, r in enumerate(ls_zero_rates)]
    ls_df = [ np.exp(-df*(i+1)) for i, df in enumerate(ls_df) ]
    # Calculate partial derivative for each payment date
    PV01_partial = (payer_swap_10y(ls_df, swap_rate) - S_0) / .0001
    ls_pv01.append(PV01_partial)

## Calculate Mean and Covariance Matrix
1. Use combined df to get mean return of each risk factor
2. Set weight matrix w

In [54]:
mean_ret = df_returns.mean().values
cov_ret = df_returns.cov().values
w = np.concatenate((np.array([1e6]*4),       # join weights for stocks and rates
                  1e8*np.array(ls_pv01)))
        # Notional * corrective to change PV01 from bp to pct * PV01
mean_1d_ret = mean_ret @ w
var_1d_ret = w @ cov_ret @ w.T

# Parametric VaR
Display possible loss as a positive number

In [55]:
var1d = stat.norm.ppf(.05, loc=mean_1d_ret, scale=np.sqrt(var_1d_ret))

print("")
print("")
print("============================================================================================================================")
print("Parametric VaR:")
print(f"Parametric VaR [1d, 95%]: {abs(var1d):,.0f}")
print("")
print("")
print("============================================================================================================================")
print(f"Mean {mean_1d_ret:,.2f}, Variance:  {var_1d_ret:,.2f}, SD:{np.sqrt(var_1d_ret):,.2f}")
print("============================================================================================================================")





Parametric VaR:
Parametric VaR [1d, 95%]: 969,948


Mean 13,663.25, Variance:  357,595,694,963.15, SD:597,993.06


# Monte Carlo Method
Reuse mean and variance from parametric period to create samples
## MC Full revaluation
0. make function to calculate pnl
1. generate samples
2. calculate PnL full revaluation\
    a. calculate initial portfolio values\
    b. calculate stock P_1 values\
    c. calculate swap P_1 values\
    d. calculate P_1 - P_0

In [58]:
# function to calculte P_1 for a sample of returns 
def P_1_calculate(samples):
    """
    Return value of swap and stock
    Parameters:
        samples: Returns in fractional terms in NxM numpy array for M = 14 (4 stocks and 10 swap fixing DF). N is no. of samples
    for stocks 
        P_1 = (1 + sample return) * P_0
    for swap
        calculate new zero rates: new zero rate = (1 + sample_return) * zero_rate_at_time_0 
        recalculate payer swap value using new discount curve
    """
    stock_P_1 = ((samples+1) * w.T)[:, :4]   # 1+R for each stock factor and multiply weight
    swap_P_1 = []

    for sample in samples:  # iterate through each sample
        ls_df = np.array(ls_zero_rates).astype(float) + sample[4:]       # calculate P_1 zero rates
        ls_df = [ np.exp(-r*(i+1)) for i, r in enumerate(ls_df) ]        # calculate P_1 DF
        # calculate S_1 and append
        swap_P_1.append(payer_swap_10y(ls_df, swap_rate))
    swap_P_1 = 1e8 * np.array(swap_P_1)

    return swap_P_1, stock_P_1

# generate samples
num_samples = 1_000_000
samples = np.random.multivariate_normal(mean_ret, 
                                        cov_ret, 
                                        num_samples)


# find P_0 initial portfolio value 
P_0 = S_0 * 1e8 + 1e6 * 4
# find P_1 day 1 portfolio value
swap_P_1, stock_P_1 = P_1_calculate(samples)
P_1 = np.concatenate([stock_P_1, 
                      swap_P_1[:, np.newaxis]],
                      axis=1)\
        .sum(axis=1)

pnl1d_full_sample = P_1 - P_0
var1d_full_mc = np.abs(np.percentile(pnl1d_full_sample, 5))

## MC Sensitiviy Analysis

In [59]:
pnl1d_sen_sample = samples@w.T
var1d_sen_mc = np.abs(np.percentile(pnl1d_sen_sample, 5))

print("")
print("")
print("============================================================================================================================")
print("Monte Carlo VaR:")
print(f"VaR [1d, 95%], Full Revaluation : {var1d_full_mc:,.0f}") 
print("")
print(f"VaR [1d, 95%], Sensitivity      : {var1d_sen_mc:,.0f}") 
print("")
print("")
print("============================================================================================================================")
print(f"Full Reval  :: Mean {pnl1d_full_sample.mean():,.2f}, \t Variance:  {pnl1d_full_sample.var():,.2f}, SD:{pnl1d_full_sample.std():,.2f}")
print(f"Sensitivity :: Mean {pnl1d_sen_sample.mean():,.2f}, \t Variance:  {pnl1d_sen_sample.var():,.2f}, SD:{pnl1d_sen_sample.std():,.2f}")
print("============================================================================================================================")



Monte Carlo VaR:
VaR [1d, 95%], Full Revaluation : 974,662

VaR [1d, 95%], Sensitivity      : 968,939


Full Reval  :: Mean 11,576.91, 	 Variance:  358,045,578,046.39, SD:598,369.10
Sensitivity :: Mean 13,625.51, 	 Variance:  357,796,189,588.99, SD:598,160.67


# Historical Var

In [60]:
hist_samples = df_returns.values
swap_P_1, stock_P_1 = P_1_calculate(hist_samples)
P_1 = np.concatenate([stock_P_1, 
                      swap_P_1[:, np.newaxis]],
                      axis=1)\
        .sum(axis=1)

# Full revaluation for Historical Var
pnl1d_full_hist_sample = P_1 - P_0
var1d_full_hist = np.abs(np.percentile(pnl1d_full_hist_sample, 5))
# Sensitivity impact for Historical Var
pnl1d_sen_hist_sample = hist_samples@w.T
var1d_sen_hist = np.abs(np.percentile(pnl1d_sen_hist_sample, 5))

print("")
print("")
print("============================================================================================================================")
print("Historical VaR:")
print(f"VaR [1d, 95%], Full Revaluation : {var1d_full_hist:,.0f}") 
print("")
print(f"VaR [1d, 95%], Sensitivity      : {var1d_sen_hist:,.0f}") 
print("")
print("")
print("============================================================================================================================")
print(f"Full Reval  :: Mean {pnl1d_full_hist_sample.mean():,.2f}, \t Variance:  {pnl1d_full_hist_sample.var():,.2f}, SD:{pnl1d_full_hist_sample.std():,.2f}")
print(f"Sensitivity :: Mean {pnl1d_sen_hist_sample.mean():,.2f}, \t Variance:  {pnl1d_sen_hist_sample.var():,.2f}, SD:{pnl1d_sen_hist_sample.std():,.2f}")
print("============================================================================================================================")



Historical VaR:
VaR [1d, 95%], Full Revaluation : 990,135

VaR [1d, 95%], Sensitivity      : 984,000


Full Reval  :: Mean 11,620.23, 	 Variance:  358,175,441,753.42, SD:598,477.60
Sensitivity :: Mean 13,663.25, 	 Variance:  356,176,664,427.58, SD:596,805.38
